####Camada Gold Delta - Criação das dimensões e fatos

In [0]:
from pyspark.sql import SparkSession

# Create a SparkSession with the required configurations for Delta Lake
spark = SparkSession.builder \
    .appName("Carga Delta") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [0]:
#Definindo variáveis com os caminhos de armazenamento
silver_path = '/FileStore/lhdw/silver/vendas'
gold_path = '/FileStore/lhdw/gold/vendas'

####Importando dados da camada silver

In [0]:
df_silver = spark.read.parquet(silver_path)


####Criando Dimensão DIM_PRODUTO

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

#Nome tabela destino
tbl_destino = "dim_produto"

#Selecionando as colunas e excluindo os registros duplicados
df_dim_produto = df_silver.select('IDProduto','Produto','Categoria')\
                          .dropDuplicates()\
                          .orderBy('IDProduto')

#Adicionando a chave primária
df_dim_produto = df_dim_produto.withColumn('sk_produto', monotonically_increasing_id()+1)

#Criando a tabela gold DIM_PRODUTO no formato Delta
df_dim_produto.write.format('delta').mode('overwrite').save(f"{gold_path}/{tbl_destino}")


####Criando dimensão DIM CATEGORIA

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

#Nome tabela destino
tbl_destino = "dim_categoria"

#Selecionando as colunas e excluindo os registros duplicados
df_dim_categoria = df_silver.select('Categoria')\
                          .distinct()\
                          .orderBy('Categoria')

#Adicionando a chave primária
df_dim_categoria = df_dim_categoria.withColumn('sk_categoria', monotonically_increasing_id()+1)

#Criando a tabela gold DIM_CATEGORIA no formato Delta
df_dim_categoria.write.format('delta').mode('overwrite').save(f"{gold_path}/{tbl_destino}")

#display(df_dim_categoria)


####Criando dimensão DIM SEGMENTO

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

#Nome tabela destino
tbl_destino = "dim_segmento"

#Selecionando as colunas e excluindo os registros duplicados
df_dim_segmento = df_silver.select('Segmento')\
                          .distinct()\
                          .orderBy('Segmento')

#Adicionando a chave primária
df_dim_segmento = df_dim_segmento.withColumn('sk_segmento', monotonically_increasing_id()+1)

#Criando a tabela gold DIM_SEGMENTO no formato Delta
df_dim_segmento.write.format('delta').mode('overwrite').save(f"{gold_path}/{tbl_destino}")

#display(df_dim_segmento)

####Criando tabela dimensão DIM FABRICANTE

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

#Nome tabela destino
tbl_destino = "dim_fabricante"

#Selecionando as colunas e excluindo os registros duplicados
df_dim_fabricante = df_silver.select('IDFabricante','Fabricante')\
                          .distinct()\
                          .orderBy('IDFabricante')

#Adicionando a chave primária
df_dim_fabricante = df_dim_fabricante.withColumn('sk_fabricante', monotonically_increasing_id()+1)

#Criando a tabela gold DIM_FABRICANTE no formato Delta
df_dim_fabricante.write.format('delta').mode('overwrite').option("overwriteSchema", "true").save(f"{gold_path}/{tbl_destino}")

#display(df_dim_fabricante)

####Criando dimensão DIM GEOGRAFIA

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

#Nome tabela destino
tbl_destino = "dim_geografia"

#Selecionando as colunas e excluindo os registros duplicados
df_dim_geografia = df_silver.select('Cidade','Estado','Regiao','Distrito','Pais','CodigoPostal')\
                          .dropDuplicates()

#Adicionando a chave primária
df_dim_geografia = df_dim_geografia.withColumn('sk_geografia', monotonically_increasing_id()+1)

#Criando a tabela gold DIM_GEOGRAFIA no formato Delta
df_dim_geografia.write.format('delta').mode('overwrite').option("overwriteSchema", "true").save(f"{gold_path}/{tbl_destino}")

#display(df_dim_geografia)

####Criando dimensão DIM CLIENTE

In [0]:
from pyspark.sql.functions import col, monotonically_increasing_id

#Nome tabela destino
tbl_destino = "dim_cliente"

#1-Extraindo clientes únicos para criar a dimensão clientes
df_dim_cliente = df_silver.select('IDCliente','Nome','Email','Cidade','Estado','Regiao','Distrito','Pais','CodigoPostal')\
                          .distinct()

#2-Fazendo join para buscar a SK_GEOGRAFIA da DIM_GEOGRAFIA
df_dim_cliente_completo = df_dim_cliente.alias('cli')\
        .join(df_dim_geografia.alias('geo'),
              (col('cli.Cidade')==col('geo.Cidade')) &
              (col('cli.Estado')==col('geo.Estado')) &
              (col('cli.Regiao')==col('geo.Regiao')) &
              (col('cli.Distrito') == col('geo.Distrito')) &
              (col('cli.Pais') == col('geo.Pais')) &
              (col('cli.CodigoPostal') == col('geo.CodigoPostal')),
              'left')\
        .select('cli.IDCliente','cli.Nome','cli.Email','geo.sk_geografia').orderBy('IDCliente')

#3-Criando a chave única de Clientes.
df_dim_cliente_completo = df_dim_cliente_completo.withColumn('sk_cliente',monotonically_increasing_id()+1)

#4-#Criando a tabela gold DIM_CLIENTE no formato Delta
df_dim_cliente_completo.write.format('delta').mode('overwrite').option("overwriteSchema", "true").save(f"{gold_path}/{tbl_destino}")

#display(df_dim_cliente_completo)

####Criando da tabela fato FT_VENDAS


In [0]:
from pyspark.sql.functions import broadcast,year, month

#Nome tabela destino
tbl_destino = 'ft_vendas'

# Juntar dados da Silver com tabelas de dimensões para obter as chaves substitutas
df_ft_vendas = df_silver.alias('s')\
    .join(broadcast(df_dim_produto.select('IDProduto','sk_produto')),'IDProduto')\
    .join(broadcast(df_dim_categoria.select('Categoria','sk_categoria')),'Categoria')\
    .join(broadcast(df_dim_segmento.select('Segmento','sk_segmento')),'Segmento')\
    .join(broadcast(df_dim_fabricante.select('IDFabricante','sk_fabricante')),'IDFabricante')\
    .join(broadcast(df_dim_cliente_completo.select('IDCliente','sk_cliente')),'IDCliente')\
    .select(col('s.Data').alias('DataVenda'),
            'sk_produto',
            'sk_categoria',
            'sk_segmento',
            'sk_fabricante',
            'sk_cliente',
            'Unidades',
            col('s.PrecoUnitario'),
            col('s.CustoUnitario'),
            col('s.TotalVendas'))

df_ft_vendas.withColumn('Ano',year('DataVenda'))\
            .withColumn('Mes',month('DataVenda'))\
            .write.format('Delta')\
            .option('MaxRecordsPerFile', 1000000)\
            .partitionBy('Ano','Mes')\
            .save(f'{gold_path}/{tbl_destino}')

#display(df_ft_vendas)


####Fazendo Limpeza da Memória

In [0]:
import gc

# Coletar lixo para liberar memória
gc.collect()

# Limpar todos os dados em cache
spark.catalog.clearCache()